In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%matplotlib inline


Transfer Learning for Computer Vision Tutorial
==============================================

These two major transfer learning scenarios look as follows:

-  **Finetuning the convnet**: Instead of random initialization, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.


In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
image_transforms  = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.Resize(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

dataset = 'microsphere2'
train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'valid')

batch_size = 16
num_classes = 173

data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])

}


train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

train_data = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)

print(train_data_size, valid_data_size)

1038 346


迁移学习
---------


这里使用ResNet-50的预训练模型。


In [5]:
resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = True

fc_inputs = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(64, 240),
    nn.LogSoftmax(dim = 1)


)
resnet50 = resnet50.to('cuda:0')
loss_func = nn.NLLLoss()
optimizer = optim.AdamW(resnet50.parameters(),lr=1e-4)

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [6]:
def train_and_valid(model, loss_function, optimizer, epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))

        model.train()

        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0

        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)

            #因为这里梯度是累加的，所以每次记得清零
            optimizer.zero_grad()

            outputs = model(inputs)

            loss = loss_function(outputs, labels)

            loss.backward()

            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            train_acc += acc.item() * inputs.size(0)

        with torch.no_grad():
            model.eval()

            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)

                loss = loss_function(outputs, labels)

                valid_loss += loss.item() * inputs.size(0)

                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                valid_acc += acc.item() * inputs.size(0)

        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size

        avg_valid_loss = valid_loss/valid_data_size
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        if best_acc < avg_valid_acc:
            best_acc = avg_valid_acc
            best_epoch = epoch + 1

        epoch_end = time.time()

        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_valid_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))

        torch.save(model, 'models/'+dataset+'_model_'+str(epoch+1)+'.pt')
    return model, history


In [7]:
num_epochs = 3000
trained_model, history = train_and_valid(resnet50, loss_func, optimizer, num_epochs)
torch.save(history, 'models/'+dataset+'_history.pt')

history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 1)
plt.savefig(dataset+'_loss_curve.png')
plt.show()

plt.plot(history[:, 2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.savefig(dataset+'_accuracy_curve.png')
plt.show()

Epoch: 1/3000
Epoch: 001, Training: Loss: 5.4714, Accuracy: 0.2890%, 
		Validation: Loss: 5.4714, Accuracy: 0.5780%, Time: 264.1641s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 2/3000
Epoch: 002, Training: Loss: 5.4448, Accuracy: 0.9634%, 
		Validation: Loss: 5.4448, Accuracy: 0.5780%, Time: 15.1994s
Best Accuracy for validation : 0.0058 at epoch 001
Epoch: 3/3000
Epoch: 003, Training: Loss: 5.3958, Accuracy: 1.1561%, 
		Validation: Loss: 5.3958, Accuracy: 1.1561%, Time: 14.6953s
Best Accuracy for validation : 0.0116 at epoch 003
Epoch: 4/3000
Epoch: 004, Training: Loss: 5.3551, Accuracy: 0.5780%, 
		Validation: Loss: 5.3551, Accuracy: 0.5780%, Time: 14.7547s
Best Accuracy for validation : 0.0116 at epoch 003
Epoch: 5/3000
Epoch: 005, Training: Loss: 5.2812, Accuracy: 1.1561%, 
		Validation: Loss: 5.2812, Accuracy: 1.1561%, Time: 14.9693s
Best Accuracy for validation : 0.0116 at epoch 003
Epoch: 6/3000
Epoch: 006, Training: Loss: 5.2094, Accuracy: 1.3487%, 
		Validation: 

Epoch: 46/3000
Epoch: 046, Training: Loss: 3.8338, Accuracy: 2.5048%, 
		Validation: Loss: 3.8338, Accuracy: 5.7803%, Time: 14.8387s
Best Accuracy for validation : 0.0578 at epoch 046
Epoch: 47/3000
Epoch: 047, Training: Loss: 3.8806, Accuracy: 3.5645%, 
		Validation: Loss: 3.8806, Accuracy: 3.7572%, Time: 14.8549s
Best Accuracy for validation : 0.0578 at epoch 046
Epoch: 48/3000
Epoch: 048, Training: Loss: 3.9417, Accuracy: 4.6243%, 
		Validation: Loss: 3.9417, Accuracy: 3.1792%, Time: 14.7495s
Best Accuracy for validation : 0.0578 at epoch 046
Epoch: 49/3000
Epoch: 049, Training: Loss: 3.7766, Accuracy: 4.1426%, 
		Validation: Loss: 3.7766, Accuracy: 6.3584%, Time: 14.9615s
Best Accuracy for validation : 0.0636 at epoch 049
Epoch: 50/3000
Epoch: 050, Training: Loss: 3.8311, Accuracy: 3.1792%, 
		Validation: Loss: 3.8311, Accuracy: 4.0462%, Time: 14.8098s
Best Accuracy for validation : 0.0636 at epoch 049
Epoch: 51/3000
Epoch: 051, Training: Loss: 3.7844, Accuracy: 4.3353%, 
		Validat

Epoch: 91/3000
Epoch: 091, Training: Loss: 3.0689, Accuracy: 5.9730%, 
		Validation: Loss: 3.0689, Accuracy: 15.0289%, Time: 14.5906s
Best Accuracy for validation : 0.1503 at epoch 091
Epoch: 92/3000
Epoch: 092, Training: Loss: 3.0917, Accuracy: 6.1657%, 
		Validation: Loss: 3.0917, Accuracy: 13.8728%, Time: 14.4090s
Best Accuracy for validation : 0.1503 at epoch 091
Epoch: 93/3000
Epoch: 093, Training: Loss: 3.1171, Accuracy: 5.3950%, 
		Validation: Loss: 3.1171, Accuracy: 14.1618%, Time: 14.4416s
Best Accuracy for validation : 0.1503 at epoch 091
Epoch: 94/3000
Epoch: 094, Training: Loss: 3.0675, Accuracy: 6.9364%, 
		Validation: Loss: 3.0675, Accuracy: 10.4046%, Time: 14.9860s
Best Accuracy for validation : 0.1503 at epoch 091
Epoch: 95/3000
Epoch: 095, Training: Loss: 3.0315, Accuracy: 7.2254%, 
		Validation: Loss: 3.0315, Accuracy: 13.0058%, Time: 14.6871s
Best Accuracy for validation : 0.1503 at epoch 091
Epoch: 96/3000
Epoch: 096, Training: Loss: 3.0298, Accuracy: 7.7071%, 
		Va

Epoch: 136, Training: Loss: 2.5678, Accuracy: 10.1156%, 
		Validation: Loss: 2.5678, Accuracy: 18.4971%, Time: 14.6020s
Best Accuracy for validation : 0.2341 at epoch 133
Epoch: 137/3000
Epoch: 137, Training: Loss: 2.6032, Accuracy: 9.1522%, 
		Validation: Loss: 2.6032, Accuracy: 18.4971%, Time: 14.6159s
Best Accuracy for validation : 0.2341 at epoch 133
Epoch: 138/3000
Epoch: 138, Training: Loss: 2.5158, Accuracy: 10.7900%, 
		Validation: Loss: 2.5158, Accuracy: 20.5202%, Time: 14.4928s
Best Accuracy for validation : 0.2341 at epoch 133
Epoch: 139/3000
Epoch: 139, Training: Loss: 2.5432, Accuracy: 10.8863%, 
		Validation: Loss: 2.5432, Accuracy: 19.0751%, Time: 14.4363s
Best Accuracy for validation : 0.2341 at epoch 133
Epoch: 140/3000
Epoch: 140, Training: Loss: 2.5643, Accuracy: 10.6936%, 
		Validation: Loss: 2.5643, Accuracy: 14.1618%, Time: 14.6970s
Best Accuracy for validation : 0.2341 at epoch 133
Epoch: 141/3000
Epoch: 141, Training: Loss: 2.5073, Accuracy: 9.4412%, 
		Validati

Epoch: 180/3000
Epoch: 180, Training: Loss: 2.2663, Accuracy: 13.9692%, 
		Validation: Loss: 2.2663, Accuracy: 24.8555%, Time: 14.6566s
Best Accuracy for validation : 0.2832 at epoch 164
Epoch: 181/3000
Epoch: 181, Training: Loss: 2.2786, Accuracy: 14.7399%, 
		Validation: Loss: 2.2786, Accuracy: 19.3642%, Time: 14.6070s
Best Accuracy for validation : 0.2832 at epoch 164
Epoch: 182/3000
Epoch: 182, Training: Loss: 2.2224, Accuracy: 13.5838%, 
		Validation: Loss: 2.2224, Accuracy: 27.4566%, Time: 14.6602s
Best Accuracy for validation : 0.2832 at epoch 164
Epoch: 183/3000
Epoch: 183, Training: Loss: 2.2711, Accuracy: 11.5607%, 
		Validation: Loss: 2.2711, Accuracy: 23.4104%, Time: 14.5567s
Best Accuracy for validation : 0.2832 at epoch 164
Epoch: 184/3000
Epoch: 184, Training: Loss: 2.2265, Accuracy: 16.6667%, 
		Validation: Loss: 2.2265, Accuracy: 21.6763%, Time: 14.6339s
Best Accuracy for validation : 0.2832 at epoch 164
Epoch: 185/3000
Epoch: 185, Training: Loss: 2.2944, Accuracy: 13.

...

Epoch: 444/3000
Epoch: 444, Training: Loss: 1.5707, Accuracy: 24.1811%, 
		Validation: Loss: 1.5707, Accuracy: 38.1503%, Time: 14.6205s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 445/3000
Epoch: 445, Training: Loss: 1.6063, Accuracy: 26.2042%, 
		Validation: Loss: 1.6063, Accuracy: 38.7283%, Time: 14.6901s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 446/3000
Epoch: 446, Training: Loss: 1.6890, Accuracy: 24.4701%, 
		Validation: Loss: 1.6890, Accuracy: 32.3699%, Time: 14.6162s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 447/3000
Epoch: 447, Training: Loss: 1.6167, Accuracy: 25.2408%, 
		Validation: Loss: 1.6167, Accuracy: 37.2832%, Time: 14.4765s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 448/3000
Epoch: 448, Training: Loss: 1.6728, Accuracy: 23.5067%, 
		Validation: Loss: 1.6728, Accuracy: 29.7688%, Time: 14.4806s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 449/3000
Epoch: 449, Training: Loss: 1.6445, Accuracy: 22.

Epoch: 488/3000
Epoch: 488, Training: Loss: 1.6590, Accuracy: 26.7823%, 
		Validation: Loss: 1.6590, Accuracy: 33.2370%, Time: 14.7352s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 489/3000
Epoch: 489, Training: Loss: 1.5675, Accuracy: 26.0116%, 
		Validation: Loss: 1.5675, Accuracy: 38.1503%, Time: 14.9540s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 490/3000
Epoch: 490, Training: Loss: 1.5578, Accuracy: 24.8555%, 
		Validation: Loss: 1.5578, Accuracy: 36.1272%, Time: 14.8545s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 491/3000
Epoch: 491, Training: Loss: 1.5270, Accuracy: 25.0482%, 
		Validation: Loss: 1.5270, Accuracy: 39.8844%, Time: 14.8521s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 492/3000
Epoch: 492, Training: Loss: 1.5971, Accuracy: 26.0116%, 
		Validation: Loss: 1.5971, Accuracy: 40.4624%, Time: 14.9149s
Best Accuracy for validation : 0.4740 at epoch 395
Epoch: 493/3000
Epoch: 493, Training: Loss: 1.5815, Accuracy: 26.

Epoch: 532, Training: Loss: 1.4860, Accuracy: 27.2640%, 
		Validation: Loss: 1.4860, Accuracy: 39.3064%, Time: 14.9218s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 533/3000
Epoch: 533, Training: Loss: 1.4815, Accuracy: 29.1908%, 
		Validation: Loss: 1.4815, Accuracy: 43.3526%, Time: 14.9103s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 534/3000
Epoch: 534, Training: Loss: 1.4669, Accuracy: 29.6724%, 
		Validation: Loss: 1.4669, Accuracy: 42.1965%, Time: 14.8747s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 535/3000
Epoch: 535, Training: Loss: 1.5283, Accuracy: 27.8420%, 
		Validation: Loss: 1.5283, Accuracy: 38.7283%, Time: 15.0003s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 536/3000
Epoch: 536, Training: Loss: 1.5297, Accuracy: 26.0116%, 
		Validation: Loss: 1.5297, Accuracy: 36.4162%, Time: 14.8438s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 537/3000
Epoch: 537, Training: Loss: 1.5541, Accuracy: 26.3006%, 
		Valida

Epoch: 576, Training: Loss: 1.8257, Accuracy: 26.3969%, 
		Validation: Loss: 1.8257, Accuracy: 28.6127%, Time: 14.7227s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 577/3000
Epoch: 577, Training: Loss: 1.4575, Accuracy: 27.9383%, 
		Validation: Loss: 1.4575, Accuracy: 41.3295%, Time: 14.9185s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 578/3000
Epoch: 578, Training: Loss: 1.4774, Accuracy: 28.1310%, 
		Validation: Loss: 1.4774, Accuracy: 39.8844%, Time: 14.7880s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 579/3000
Epoch: 579, Training: Loss: 1.4684, Accuracy: 29.5761%, 
		Validation: Loss: 1.4684, Accuracy: 43.9306%, Time: 14.9228s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 580/3000
Epoch: 580, Training: Loss: 1.4544, Accuracy: 29.8651%, 
		Validation: Loss: 1.4544, Accuracy: 44.2197%, Time: 14.8531s
Best Accuracy for validation : 0.4855 at epoch 528
Epoch: 581/3000
Epoch: 581, Training: Loss: 1.6528, Accuracy: 30.2505%, 
		Valida

KeyboardInterrupt: 